In [1]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        if i < 500:
            i = i +1
            X.append(row)

y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        if j < 500:
            j = j +1
            y.append(row.replace("\n",""))


In [2]:
X = np.array(X)
y = np.array(y)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

In [4]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))

test = np.asarray(test)
test_dtm = vect.transform(test)

In [5]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

(50000, 1)
(50000,)


In [8]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, f1_score, jaccard_score, classification_report

models = [
    MLPClassifier(solver = 'adam', activation = 'logistic', learning_rate = 'adaptive')
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    predict_label = model.predict(test_dtm)
    acc = accuracy_score(predict_label,test_label)
    f1 = f1_score(predict_label, test_label, average = 'weighted')
    cm = multilabel_confusion_matrix(test_label,predict_label)
    jaccard = jaccard_score(test_label, predict_label, average='micro')
    print(classification_report(test_label, predict_label))
    print ("la matrice de confusion : ")
    print(cm)
    entries.append((model_name, acc, f1, jaccard))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'accuracy', 'f1', 'jaccard'])

/home/julien/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.24      0.57      0.34     10798
           1       0.13      0.10      0.11      4830
          10       0.05      0.01      0.01      1432
          11       0.27      0.08      0.12      1949
          12       0.18      0.05      0.08      1265
          13       0.02      0.00      0.00      1114
          14       0.05      0.00      0.01      1306
          15       0.06      0.03      0.04      1244
          16       0.02      0.00      0.01      1153
          17       0.62      0.12      0.21      1545
          18       0.14      0.01      0.02      2417
          19       0.05      0.00      0.00      1010
           2       0.15      0.28      0.20      4534
           3       0.08      0.08      0.08      2605
           4       0.27      0.15      0.19      3716
           5       0.05      0.05      0.05      1613
           6       0.06      0.03      0.04      1996
           7       0.14    

In [9]:
print (cv_df)

      model_name  accuracy        f1   jaccard
0  MLPClassifier   0.18888  0.234367  0.104289


In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

# Adam works better ; Logistic works better
parameters = {'solver':('lbfgs', 'adam'), 'activation':('relu', 'logistic'), 'alpha':[0.0001, 0.00001, 0.001], 
              'learning_rate':('adaptive'), }

mlp = MLPClassifier()

clf = GridSearchCV(mlp, parameters, cv=3, n_jobs=-1)

clf.fit(X_dtm, y)
                            
print(clf.cv_results_)